In [7]:
import json
import math
import numpy as np
import pandas as pd
# from flask import Flask, jsonify, request
fname = "slay-plan-example.json"
conceptName = "Concept 1 (2).json"
with open(fname, "r") as json_file:
    f=json_file.read()
    data = json.loads(f)
with open(conceptName, "r") as json_file:
    f=json_file.read()
    concept_dict = json.loads(f)
len(data)
#level 1
for i in data["children"][0]:
    print(i)
# for i in data["elements"][0]:
#     print (i)
#         print(j["name"])

@class
id
name
properties
type
children
template
path
people
equipment
consumables
activity
sowForTask
startDate
endDate
duration
transitPath
location
element
predecessors
transitDistance


In [8]:
data["children"][1]["activity"]["type"]

'Demobilisation'

In [9]:
import utm
utm.from_latlon(51.2, 7.5)
projectLocationWgsLatLon=(51,1)

In [13]:
utm.to_latlon(340000, 5710000, 32, 'U')
def getPathWgsFromWrongOc(OcPoints):
    WgsPoints=[]
    for point in OcPoints:
        WgsPoints.append({"lon":point["x"], "lat":point["y"]})
    return WgsPoints

def getPathOcFromWgs(WgsPoints):
    OcPoints=[]
    east0, nort0, zone0, zoneS0=utm.from_latlon(projectLocationWgsLatLon[0],projectLocationWgsLatLon[1])
    for point in WgsPoints:
        east, nort, zone, zoneS = utm.from_latlon(point["lat"], point["lon"])
        OcPoints.append({"x":east-east0, "y":nort-nort0, "z":0})
    return OcPoints
        
def getPathWgsFromOc(OcPoints):
    WgsPoints=[]
    east0, nort0, zone0, zoneS0=utm.from_latlon(projectLocationWgs)
    for point in OcPoints:
        lat, lon = utm.to_latlon(point["x"]+east0, point["y"]+nort0, zone0, zoneS0)
        WgsPoints.append({"lon":lon, "lat":lat})
    return WgsPoints

In [14]:
def calKp(points):
    KPs=[]
    currentKP=0
    segment_l=0
    for index, point in enumerate(points):
        if index>0:
            segment_l=math.sqrt((point["x"]-points[index-1]["x"])**2+\
                     (point["y"]-points[index-1]["y"])**2+\
                     (point["z"]-points[index-1]["z"])**2)
#         print(path,segment_l)
            currentKP+=segment_l
        KPs.append(currentKP)
    return(KPs)

def cal2dLength(points):
    currentKP=0
    segment_l=0
    for index, point in enumerate(points):
        if index>0:
            segment_l=math.sqrt((point["x"]-points[index-1]["x"])**2+\
                     (point["y"]-points[index-1]["y"])**2)
#         print(path,segment_l)
            currentKP+=segment_l
    return(currentKP)        
    
def getPointAtDistance(p2,p1,distFromP2):
    v=(p2["x"]-p1["x"],p2["y"]-p1["y"],p2["z"]-p1["z"])
    v=np.array(v)
    segment_l=math.sqrt((p2["x"]-p1["x"])**2+\
                     (p2["y"]-p1["y"])**2+\
                     (p2["z"]-p1["z"])**2)
    newPoint=v/segment_l*(segment_l-distFromP2)
    newPointGlobal=newPoint[0]+p1["x"],newPoint[1]+p1["y"],newPoint[2]+p1["z"]
    return({"x":newPointGlobal[0],"y":newPointGlobal[1],"z":newPointGlobal[2]})

def getPointsByPathAndKp(path,fromKP,toKP):
    for element in concept_dict["elements"]:
        if element["path"]==path:
            for prop in element["properties"]:
                if prop["name"]=="points":
                    points=prop["value"]
#     print(points)
    tpath=[]
    currentKP=0
    segment_l=0
    flag=True
    for index, point in enumerate(points):
        if index>0:
            segment_l=math.sqrt((point["x"]-points[index-1]["x"])**2+\
                     (point["y"]-points[index-1]["y"])**2+\
                     (point["z"]-points[index-1]["z"])**2)
#         print(path,segment_l)
        currentKP+=segment_l
        if fromKP>0 and fromKP>currentKP:
            continue
        elif fromKP>0 and fromKP<currentKP and flag:
            difference=currentKP-fromKP
            firstPoint=getPointAtDistance(point, points[index-1], difference)
            tpath.append(firstPoint)
            tpath.append(point)
            flag=False
        elif toKP==-1 or toKP>=currentKP:
#             point["kp"]=currentKP
            tpath.append(point)
        elif toKP<currentKP:
            difference=currentKP-toKP
            lastPoint=getPointAtDistance(point, points[index-1], difference)
            tpath.append(lastPoint)
            toKP=100000000
            break
#     print(path,currentKP)
    return tpath

def getPointsByPath(path):
    for element in concept_dict["elements"]:
        if element["path"]==path:
            for prop in element["properties"]:
                if prop["name"]=="points":
                    return prop["value"]
                
def getpath(nested_dict, value, prepath=()):
    for k, v in nested_dict.items():
        path = prepath + (k,)
        if v == value: # found value
            return path
        elif isinstance(v, list):
            for item in v:
                if not isinstance(item, dict):
                    continue
#         elif hasattr(v, 'items'): # v is a dict
                else: 
                    p = getpath(item, value, path) # recursive call
                    if p is not None:
                        return p
            
print(getpath(concept_dict,"9dcfb73d-f73d-4bb9-aae7-6cda4f1d3155"))

('elements', 'properties', 'id')


In [15]:


newDic=[]
def getDicForAnimationFromVesselChildren(vesselChildrenList):
    for datadict in vesselChildrenList:
        el_length=None
        fromKP=None
        toKP=None
        linePoints=None
        lineKp=None
        if "activity" in datadict and datadict["activity"]!=None:
            if datadict["element"]==None:
                element=None
                _tranzitPath=datadict["transitPath"]
                tranzitPathWgs = getPathWgsFromWrongOc(_tranzitPath)
                tranzitPathOc = getPathOcFromWgs(tranzitPathWgs)
                el_length = cal2dLength(tranzitPathOc)
            else:
                element=datadict["element"]["name"]
                for prop in datadict["properties"]:
                    if prop["name"]== "Element Length":
                        el_length = prop["value"]
                    if prop["name"]== "from":
                        fromKP = prop["value"]
                    if prop["name"]== "to":
                        toKP = prop["value"]
                if datadict["properties"]==[]:
                        tranzitPathOc=None
                        tranzitPathWgs=None
                else:
                    tranzitPathOc=getPointsByPathAndKp(datadict["element"]["path"],fromKP,toKP)
                    tranzitPathWgs= getPathWgsFromOc(tranzitPathOc)
                    linePoints = getPointsByPath(datadict["element"]["path"])
#                 print(datadict["element"]["path"],tranzitPath)

            newDic.append({
                "name":datadict["name"], 
                "path":datadict["path"], 
                "activity":datadict["activity"]["type"],
                "startDate":datadict["startDate"],
                "endDate":datadict["endDate"],
                "element":element,
                "tranzitLength":el_length,
                "fromKP":fromKP,
                "toKP":toKP,
                "tranzitPathWgs":tranzitPathWgs,
                "tranzitPathOc":tranzitPathOc
                
            })
    return newDic
    
stack=[]
final_dict={}
def do_walk(datadict):

    if isinstance(datadict, dict):
        for key, value in datadict.items():
            stack.append(key)
            if isinstance(value, dict) and len(value.keys()) == 0:
                final_dict["/".join(stack)] = "EMPTY_DICT"
            if isinstance(value, list) and len(value) == 0:
                final_dict["/".join(stack)] = 'EMPTY_LIST'
            if isinstance(value, dict):
                do_walk2(value)
            if isinstance(value, list):
                do_walk2(value)
            if isinstance(value, str):
                final_dict["/".join(stack)] = value
            
            stack.pop()

    if isinstance(datadict, list):
        n = 0
        for key in datadict:
            stack.append(str(n))
            n = n + 1
            if isinstance(key, dict):
                do_walk2(key)
            if isinstance(key, list):
                do_walk2(key)
            if isinstance(key, str):
                final_dict["/".join(stack)] = key
            stack.pop()
    return final_dict

vesselJsonForAnimation = getDicForAnimationFromVesselChildren(data["children"])

TypeError: from_latlon() missing 1 required positional argument: 'longitude'

In [33]:
df=pd.DataFrame(vesselJsonForAnimation)
df.startDate=pd.to_datetime(df['startDate'],unit='ms')
df.endDate=pd.to_datetime(df['endDate'],unit='ms')
df.sort_values("startDate",inplace=True)
df.reset_index(drop=True, inplace=True)
df

,name,path,activity,startDate,endDate,element,tranzitLength,fromKP,toKP,tranzitPath
0,Mobilisation,/OSC-P010-C008-S002/slay/Trip 1 Pipelay/Interi...,Mobilisation,2021-10-27 20:37:13.189,2021-10-28 20:37:13.189,None,0.000000,NaN,NaN,[]
1,TRANSIT : Aberdeen - WI_CS line_001,/OSC-P010-C008-S002/slay/Trip 1 Pipelay/TRANSI...,Transit,2021-10-28 20:37:13.189,2021-10-29 10:48:13.189,None,3.530964,NaN,NaN,"[{'x': -2.0928094, 'y': 57.148243, 'z': 0.0}, ..."
2,Initiation WI_CS line_001,/OSC-P010-C008-S002/slay/Trip 1 Pipelay/Pipela...,Pipelay,2021-10-29 10:48:13.189,2021-10-30 10:48:13.189,WI_CS line_001,NaN,NaN,NaN,None
3,Lay WI_CS line_001 From KP: 0.00 To KP: 2.83,/OSC-P010-C008-S002/slay/Trip 1 Pipelay/Pipela...,Pipelay,2021-10-30 10:48:13.189,2021-10-31 06:08:13.189,WI_CS line_001,2827.934700,0.000000,-1.000000,"[{'x': -8103.3115, 'y': 4181.8975, 'z': -146.4..."
4,Termination WI_CS line_001,/OSC-P010-C008-S002/slay/Trip 1 Pipelay/Pipela...,Pipelay,2021-10-31 06:08:13.189,2021-11-01 06:08:13.189,WI_CS line_001,NaN,NaN,NaN,None
5,TRANSIT : WI_CS line_001 - WI_CS line_002,/OSC-P010-C008-S002/slay/Trip 1 Pipelay/TRANSI...,Transit,2021-11-01 06:08:13.189,2021-11-01 07:28:13.189,None,0.363977,NaN,NaN,"[{'x': 0.8932406, 'y': 58.083176, 'z': 0.0}, {..."
6,Initiation WI_CS line_002,/OSC-P010-C008-S002/slay/Trip 1 Pipelay/Pipela...,Pipelay,2021-11-01 07:28:13.189,2021-11-02 07:28:13.189,WI_CS line_002,NaN,NaN,NaN,None
7,Lay WI_CS line_002 From KP: 0.00 To KP: 13.64,/OSC-P010-C008-S002/slay/Trip 1 Pipelay/Pipela...,Pipelay,2021-11-02 07:28:13.189,2021-11-06 04:42:13.189,WI_CS line_002,13644.781462,0.000000,13644.781462,"[{'x': 12149.414, 'y': -8038.7803, 'z': -121.1..."
8,Lay WI_CS line_002 From KP: 13.64 To KP: 27.17,/OSC-P010-C008-S002/slay/Trip 1 Pipelay/Pipela...,Pipelay,2021-11-06 04:42:13.189,2021-11-10 01:07:13.189,WI_CS line_002,13527.077038,13644.781462,-1.000000,"[{'x': 5279.0299357458925, 'y': 2239.222068032..."
9,Termination WI_CS line_002,/OSC-P010-C008-S002/slay/Trip 1 Pipelay/Pipela...,Pipelay,2021-11-10 01:07:13.189,2021-11-11 01:07:13.189,WI_CS line_002,NaN,NaN,NaN,None


In [29]:
df.to_csv("testPlan.csv")

In [6]:
df

NameError: name 'df' is not defined